## Comprehensive Regression Analysis

### Estimating Crown Closure in TFL 38 Using VRI Data

#### 1. Problem Definition:
Crown closure is a key forest metric representing the proportion of ground covered by tree canopies. It influences habitat suitability, light availability, and stand productivity. This project aims to develop a regression-based machine learning model to predict crown closure using key forest inventory attributes from Vegetation Resources Inventory (VRI) data. This project follows a structured machine learning workflow, including data preprocessing, exploratory analysis, feature selection, model training, and evaluation, to opimize model performance.

#### 2. Data Description and Preprocessing
The dataset used in this project is sourced from the British Columbia Vegetation Resources Inventory (VRI), which provides stand-level forest attributes collected through remote sensing and field surveys. The target variable for this analysis is crown closure (%), which represents the proportion of the forest floor covered by tree canopies. The predictor variables selected for the model are basal area (m²/ha), VRI live stems per hectare, projected age (years), projected height (m), and whole stem biomass per hectare (Mg/ha). 

The original dataset contains VRI data for the whole province of British Columbia. I decided to filter my data in ArcGIS Pro to exclude all projects except for TFL 38, a tree farm licence located northwest of Squamish, BC. I also filtered out all unnecessary fields. I then exported my filtered data to a new feature class named "tfl_38_VRI_cleaned".


2.1 Load Required Packages:

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error


2.2 Load and Inspect Data:

In [ ]:
# Load the shapefile (update user specific file path)
shapefile_path = r"C:\Users\ryanj\Desktop\project_1\tfl_38_VRI_cleaned\tfl_38_VRI_cleaned.shp"
tfl_38_VRI = gpd.read_file(shapefile_path)

# Display the first few rows
print(tfl_38_VRI.head())

# Check the column names and data types
print(tfl_38_VRI.info())

# Check for missing values
missing_values = tfl_38_VRI.isnull().sum()

# Print missing values or a message if none
if missing_values.sum() == 0:
    print("No missing values")
else:
    print(missing_values)

There are no missing values, but I noticed that some of the column names are incorrect. I also want to drop some of the unnecessary variables.

In [ ]:
# Verifying column names
print(tfl_38_VRI.columns)

2.3 Drop Unnecessary Variables:

In [ ]:
# List of columns to keep
cols_to_keep = ['FEATURE_ID', 'CROWN_CLOS', 'BASAL_AREA', 'VRI_LIVE_S', 
                'PROJ_AGE_1', 'PROJ_HEIGH', 'WHOLE_STEM', 'Shape_Leng', 
                'Shape_Area', 'geometry']

# Create a new DataFrame with only the specified columns
VRI_cleaned = tfl_38_VRI[cols_to_keep]

# Check the first few rows to confirm
print(VRI_cleaned.head())


The following columns are not named correctly:
- `CROWN_CLOS` should be: `CROWN_CLOSURE`
- `VRI_LIVE_S` should be: `VRI_LIVE_STEMS`
- `PROJ_HEIG` should be: `PROJ_HEIGHT`
- `WHOLE_STEM` should be: `WHOLE_STEMS`
- `Shape_Leng` should be: `Shape_Length`

In [ ]:
# Fix variable names
VRI_cleaned = VRI_cleaned.rename(columns={
    'CROWN_CLOS': 'CROWN_CLOSURE',
    'VRI_LIVE_S': 'VRI_LIVE_STEMS',
    'PROJ_HEIGH': 'PROJ_HEIGHT_1',  
    'WHOLE_STEM': 'WHOLE_STEM_BIOMASS',
    'Shape_Leng': 'Shape_Length'
})

# Print the updated variable names to verify changes
print(VRI_cleaned.columns)

2.4 Basic Statistics:

In [ ]:
# Basic statistics
print("\n--- Basic Statistics ---")
print(VRI_cleaned.describe())

# Check the Coordinate Reference System (CRS)
print("\n--- CRS ---")
print(VRI_cleaned.crs)

print(VRI_cleaned.columns)

# Verify spatial data integrity:
VRI_cleaned.plot(figsize=(10, 10), edgecolor="black")
plt.title("TFL 38 VRI - Map")
plt.show()

Now that my data is cleaned and verified, I will move on to exploratory data analysis.

#### 3. Exploratory Data Analysis:

In [ ]:
# Check data type
column_data_types = VRI_cleaned.dtypes
print(column_data_types)

3.1 Explore Variable Properties and Relationships:

In [ ]:
# Compute summary statistics
summary_stats = VRI_cleaned.describe()
print("\n--- Summary Statistics ---")
print(summary_stats)

# Create scatter plot matrix
sns.pairplot(VRI_cleaned)
plt.show()

In [ ]:
# Select numeric columns
numeric_columns = VRI_cleaned.select_dtypes(include=['float64', 'int64', 'int32'])

# Compute the correlation matrix for numeric columns
correlation_matrix = numeric_columns.corr()

# Display the full correlation matrix
print("\n--- Correlation Matrix ---")
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(correlation_matrix)

3.2 Correlation coefficient results:

Strong Correlations:
- **Basal Area & Whole Stem Biomass (0.9065)** – Stands with a higher basal area tend to have greater whole stem biomass.
- **Projected Height & Whole Stem Biomass (0.9166)** – Taller trees generally contribute to greater biomass.
- **Basal Area & Projected Height (0.7981)** – More basal area is associated with taller trees.
- **Shape Length & Shape Area (0.8867)** –  Greater length correlates with its area.

Moderate Correlations:
- **Crown Closure & Basal Area (0.7226)** – Denser crown cover is linked to higher basal area.
- **Crown Closure & Whole Stem Biomass (0.6378)** – More crown closure generally leads to more biomass.
- **Projected Age & Projected Height (0.4284)** – Older stands tend to be taller, but the correlation is not strong.

In [ ]:
# Plot histograms with KDE for Crown Closure vs Basal Area, and Crown Closure vs Whole Stem Biomass
crown_closure = VRI_cleaned['CROWN_CLOSURE']
basal_area = VRI_cleaned['BASAL_AREA'] 
whole_stem_biomass = VRI_cleaned['WHOLE_STEM_BIOMASS']

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Crown Closure vs Basal Area
sns.histplot(crown_closure, kde=True, color='salmon', bins=30, ax=axes[0], label='Crown Closure')
sns.histplot(basal_area, kde=True, color='bisque', bins=30, ax=axes[0], label='Basal Area')
axes[0].set_title("Crown Closure vs Basal Area")
axes[0].set_xlabel("Values")
axes[0].set_ylabel("Frequency")
axes[0].legend()

# Crown Closure vs Whole Stem Biomass
sns.histplot(crown_closure, kde=True, color='orange', bins=30, ax=axes[1], label='Crown Closure')
sns.histplot(whole_stem_biomass, kde=True, color='chocolate', bins=30, ax=axes[1], label='Whole Stem Biomass')
axes[1].set_title("Crown Closure vs Whole Stem Biomass")
axes[1].set_xlabel("Values")
axes[1].set_ylabel("Frequency")
axes[1].legend()

plt.tight_layout()
plt.show()

Distribution:
- **Crown Closure vs Basal Area** - Normal Distribution
- **Crown Closure vs Whole Stem Biomass** - Right-Skewed Distribution

#### 4. Feature Engineering:

4.1 Data Transformation:

In [ ]:
# Check skewness of the variables
skewed_features = VRI_cleaned.skew()
print(skewed_features)

The output of the above code returned features from the "geometry" variable. The output confirms that the 'geometry' variable contains valid polygon features. Since transformations are typically applied to numerical attributes to correct skewness, and the geometry itself does not require such adjustments, variable transformation is not necessary in this case.

4.2 Dimensionality Reduction:

Principal Component Analysis (PCA) helps reduce the number of features while retaining as much variance as possible. 



In [ ]:
# Select only numerical features (excluding 'CROWN_CLOSURE' and 'geometry')
num_features = VRI_cleaned.drop(columns=['CROWN_CLOSURE', 'geometry'])

# Standardize the data
scaler = StandardScaler()
num_features_scaled = scaler.fit_transform(num_features)

# Apply PCA
pca = PCA()
principal_components = pca.fit_transform(num_features_scaled)

# Explained variance plot
plt.figure(figsize=(8, 5))
plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o', linestyle='--')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Explained Variance by Components')
plt.grid()
plt.show()

# Determine how many components explain 95% of the variance
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
n_components = np.argmax(cumulative_variance >= 0.95) + 1  # Get the first index where variance ≥ 95%

print(f"Optimal number of components to retain 95% variance: {n_components}")

# Apply PCA with the selected number of components
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(num_features_scaled)

# Convert back to DataFrame
df_pca = pd.DataFrame(principal_components, columns=[f'PC{i+1}' for i in range(n_components)])
df_pca['CROWN_CLOSURE'] = VRI_cleaned['CROWN_CLOSURE']  # Retain target variable

print(f"New dataset shape after PCA: {df_pca.shape}")

4.3 Data Augmentation:

I will now enhance the dataset by artificially increasing the number of samples to improve model performance. To achieve this, I will apply Gaussian Noise Injection, subtly perturbing numerical features to generate synthetic samples.

In [ ]:
# Define the number of synthetic samples (20% of original dataset)
num_synthetic_samples = int(0.2 * len(df_pca))

# Select numerical features excluding target variable
X_numeric = df_pca.drop(columns=['CROWN_CLOSURE'])

# Generate synthetic data by adding Gaussian noise
np.random.seed(42)
noise = np.random.normal(loc=0, scale=0.01, size=(num_synthetic_samples, X_numeric.shape[1]))  # Small noise
synthetic_samples = X_numeric.sample(n=num_synthetic_samples, replace=True).values + noise

# Convert to DataFrame
df_synthetic = pd.DataFrame(synthetic_samples, columns=X_numeric.columns)

# Assign synthetic target values from sampled real values
df_synthetic['CROWN_CLOSURE'] = df_pca['CROWN_CLOSURE'].sample(n=num_synthetic_samples, replace=True).values

# Append to original dataset
df_augmented = pd.concat([df_pca, df_synthetic], ignore_index=True)

print(f"Dataset shape after augmentation: {df_augmented.shape}")

#### 5. Model Selection:

5.1 Describe the Regression Algorithms to be Used:

a) **Linear Regression**
-   **Strengths:** Simple, interpretable, fast to train.
-   **Weaknesses:** Assumes linearity, sensitive to outliers.
-   **Assumptions:** Linearity, homoscedasticity, normality of errors.
-   **Limitations:** Poor performance with complex, non-linear relationships.

b) **Random Forest Regression**
-   **Strengths:** Handles non-linear data, robust to overfitting.
-   **Weaknesses:** Computationally expensive, less interpretable.
-   **Assumptions:** Uses ensemble decision trees to improve predictions.
-   **Limitations:** Performance can degrade with irrelevant features.

c) **Support Vector Regression (SVR)**
-   **Strengths:** Good for high-dimensional spaces, handles non-linearity.
-   **Weaknesses:** Sensitive to kernel choice, computationally expensive.
-   **Assumptions:** Relies on kernel functions for non-linear modeling.
-   **Limitations:** Struggles with noisy data and large datasets.

d) **K-Nearest Neighbors (KNN) Regression**
-   **Strengths:** Simple, non-parametric, captures local patterns.
-   **Weaknesses:** Slow predictions, sensitive to irrelevant features.
-   **Assumptions:** Similar instances have similar target values.
-   **Limitations:** Struggles with high-dimensional data, computationally expensive.

e) **Gradient Boosting Regression**
-   **Strengths:** Handles non-linear relationships, high predictive accuracy.
-   **Weaknesses:** Computationally expensive, requires careful tuning.
-   **Assumptions:** Uses weak learners to iteratively improve the model.
-   **Limitations:** Can overfit without proper tuning, less interpretable.

5.2 Select Most Suitable Regression Algorithms

Based on my project and the above model descriptions, the three most suitable algorithms are:

a) **Random Forest Regression**

**Reason:** This model is well-suited for handling complex, non-linear relationships in data and is robust to overfitting. Additionally, it can handle a variety of features, including numerical and categorical, which is important in geospatial data analysis.

b) **Support Vector Regression (SVR)**

**Reason:** SVR is effective for high-dimensional data and can capture non-linear relationships. It's also useful when there's a need to generalize well to unseen data, which can be crucial for model accuracy.

c) **Gradient Boosting Regression**

**Reason:** This algorithm is known for its high predictive accuracy, especially with complex datasets. Gradient Boosting can handle non-linearities well and provides a good balance between bias and variance when tuned properly.

#### 6. Model Training:

6.1 Create Training/Testing Sets (Using an 80/20 Split)

In [ ]:
# Split the data into features (X) and target variable (y)
X = df_pca.drop(columns='CROWN_CLOSURE')  # Features after PCA
y = df_pca['CROWN_CLOSURE']  # Target variable

# Split into training and testing sets (80/20 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

6.2 Train the Regression Models (with Cross-Validation)

In [ ]:
# Define the models
rf_model = RandomForestRegressor(random_state=42)
svr_model = SVR()
gbr_model = GradientBoostingRegressor(random_state=42)

# Train the models with cross-validation (using 5 folds)
rf_cv = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
svr_cv = cross_val_score(svr_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
gbr_cv = cross_val_score(gbr_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

# Output the mean cross-validation scores
print(f"Random Forest CV MSE: {np.mean(rf_cv)}")
print(f"SVR CV MSE: {np.mean(svr_cv)}")
print(f"Gradient Boosting CV MSE: {np.mean(gbr_cv)}")

Interpreting the Results:

-   **MSE Significance**: A lower MSE indicates better model performance, as it means the model's predictions are closer to the true values. A model with a less negative MSE is preferred.

-   **Comparison**: Random Forest has the lowest MSE, suggesting it performs better than the other two models in terms of minimizing prediction errors. SVR has the highest MSE, indicating it is performing the worst among the three models. Gradient Boosting performed better than SVR but worse than Random Forest.

6.3 Tune the Models

In [ ]:
# Define parameter distributions for each model

# Random Forest parameter distribution
rf_param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# SVR parameter distribution
svr_param_dist = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Gradient Boosting parameter distribution
gbr_param_dist = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 4],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# Perform RandomizedSearchCV for each model

# Random Forest Randomized Search
rf_random_search = RandomizedSearchCV(
    estimator=rf_model,
    param_distributions=rf_param_dist,
    n_iter=10,  # Limits the number of random samples
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,  # Uses all available CPU cores
    random_state=42
)

# SVR Randomized Search
svr_random_search = RandomizedSearchCV(
    estimator=svr_model,
    param_distributions=svr_param_dist,
    n_iter=10,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42
)

# Gradient Boosting Randomized Search
gbr_random_search = RandomizedSearchCV(
    estimator=gbr_model,
    param_distributions=gbr_param_dist,
    n_iter=10,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42
)

# Fit models
print("Tuning Random Forest...")
rf_random_search.fit(X_train, y_train)
print("Best Random Forest Params:", rf_random_search.best_params_)

print("Tuning SVR...")
svr_random_search.fit(X_train, y_train)
print("Best SVR Params:", svr_random_search.best_params_)

print("Tuning Gradient Boosting...")
gbr_random_search.fit(X_train, y_train)
print("Best Gradient Boosting Params:", gbr_random_search.best_params_)


In [ ]:
# Train models with best parameters
rf_best = RandomForestRegressor(
    n_estimators=100, 
    min_samples_split=2,
    min_samples_leaf=1,
    max_depth=None, 
    max_features='sqrt', 
    random_state=42
)

svr_best = SVR(
    kernel='rbf',
    gamma='auto',
    C=100
)

gbr_best = GradientBoostingRegressor(
    n_estimators=200,
    min_samples_split=2,
    min_samples_leaf=2,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)

# Fit the models
rf_best.fit(X_train, y_train)
svr_best.fit(X_train, y_train)
gbr_best.fit(X_train, y_train)

# Predict on test set
rf_preds = rf_best.predict(X_test)
svr_preds = svr_best.predict(X_test)
gbr_preds = gbr_best.predict(X_test)

# Evaluate performance using Mean Squared Error
rf_mse = mean_squared_error(y_test, rf_preds)
svr_mse = mean_squared_error(y_test, svr_preds)
gbr_mse = mean_squared_error(y_test, gbr_preds)

print(f"Final Random Forest MSE: {rf_mse}")
print(f"Final SVR MSE: {svr_mse}")
print(f"Final Gradient Boosting MSE: {gbr_mse}")

#### 7. Model Evaluation:

7.1 Compute Root Mean Squared Error (RMSE) Values

In [ ]:
# Compute RMSE values from MSE
rf_rmse = np.sqrt(67.81497609147608)
svr_rmse = np.sqrt(85.1698000519334)
gbr_rmse = np.sqrt(76.12485597094468)

# Store RMSE values
rmse_values = {
    "Random Forest": rf_rmse,
    "SVR": svr_rmse,
    "Gradient Boosting": gbr_rmse
}

# Print RMSE values
for model, rmse in rmse_values.items():
    print(f"{model} RMSE: {rmse:.4f}")

In [ ]:
# Create a bar plot for RMSE comparison
plt.figure(figsize=(8, 5))
plt.bar(rmse_values.keys(), rmse_values.values(), color=['blue', 'red', 'green'])
plt.xlabel("Models")
plt.ylabel("RMSE")
plt.title("Model Comparison Based on RMSE")
plt.ylim(min(rmse_values.values()) - 2, max(rmse_values.values()) + 2)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Display the RMSE values on top of each bar
for i, value in enumerate(rmse_values.values()):
    plt.text(i, value + 0.5, f"{value:.2f}", ha='center', fontsize=12)

plt.show()


In [ ]:
# Calculate the standard deviation of y_test (target variable)
std_y_test = np.std(y_test)

# Print the standard deviation of y_test
print("\n--- Target Variable Standard Deviation ---")
print(std_y_test)

# Compare RMSE to Standard Deviation
print("\n--- Comparison between RMSE and Standard Deviation ---")
for model_name, rmse in rmse_values.items():
    if rmse < std_y_test:
        print(f"The RMSE for {model_name} ({rmse:.2f}) is lower than the standard deviation of the target variable ({std_y_test:.2f}), indicating good model performance.")
    elif rmse == std_y_test:
        print(f"The RMSE for {model_name} ({rmse:.2f}) is equal to the standard deviation of the target variable ({std_y_test:.2f}), suggesting that the model is performing similarly to predicting the mean value.")
    else:
        print(f"The RMSE for {model_name} ({rmse:.2f}) is higher than the standard deviation of the target variable ({std_y_test:.2f}), suggesting that the model is not performing well.")


7.2 Select the Best Performing Model

In [ ]:
# Select the model with the lowest RMSE
best_model = min(rmse_values, key=rmse_values.get)
print(f"The best-performing model is: {best_model} with an RMSE of {rmse_values[best_model]:.4f}")

#### 8. Interpretation of Results:

##### Model Performance Evaluation
The model performance was assessed using Mean Squared Error (MSE) and Root Mean Squared Error (RMSE) both before and after hyperparameter tuning. The post-tuning results are as follows:

- **Random Forest:**  
  - MSE = **67.81**  
  - RMSE = **8.23**  

- **Support Vector Regression (SVR):**  
  - MSE = **85.17**  
  - RMSE = **9.23**  

- **Gradient Boosting:**  
  - MSE = **76.12**  
  - RMSE = **8.72**  

Among the three models, Random Forest performed the best, achieving the lowest RMSE (8.23), which suggests it had the most accurate predictions for crown closure. SVR had the highest error, indicating it struggled more with the dataset.

##### Model Performance vs. Expectations
The models performed reasonably well. An RMSE of 8-9 means that the predicted crown closure values deviate from the actual values by about 8-9 percentage points on average. Given the complexity of forest structure, this level of error seems acceptable.

##### Deployment Feasibility
Based on these results, Random Forest appears to be the most viable model for deployment. If crown closure predictions were being used for habitat modeling or forestry planning, further validation would be necessary.

##### Improvements for Better Performance
To improve model accuracy, the following steps could be taken:

- **Hyperparameter Fine-tuning:** Further optimization, especially for Gradient Boosting, might yield better results.  
- **Alternative Models:** Trying other models such as neural networks or ensemble stacking could enhance performance.  

##### Effect of Hyperparameter Tuning
MSE Results before and after tuning:

- **Random Forest:**  
  - Before tuning: MSE = **-68.29**  
  - After tuning: MSE = **67.81**  

- **Support Vector Regression (SVR):**  
  - Before tuning: MSE = **-111.12**  
  - After tuning: MSE = **85.17**  

- **Gradient Boosting:**  
  - Before tuning: MSE = **-90.09**  
  - After tuning: MSE = **76.12**  

**Summary:**
  - Hyperparameter tuning has successfully improved model performance, leading to better MSE values for all three models.
  - Random Forest was the best performer after tuning, followed by Gradient Boosting, and then SVR.